In [1]:
import numpy as np

In [2]:
class Atom:
    def __init__(self,x,y,z,element,charge,notused,fx,fy,fz):
        self.x=float(x)
        self.y=float(y)
        self.z=float(z)
        self.element=str(element)
        self.charge=float(charge)
        self.notused=int(float(notused))
        self.fx=float(fx)
        self.fy=float(fy)
        self.fz=float(fz)
    
    def scan_vicinity(self,config,cutoff,boxl):
        pivot=np.array([self.x,self.y,self.z])
        config=np.array(config)
        for ind,i in enumerate(config):
            #print(str(ind)+" element= "+i.element,type(i.notused))
            if(i.notused!=0):
                pass
            elif(i.element=="O"):
                i_vector=np.array([i.x,i.y,i.z])
                #print("O at"+str(ind))
                direction=pivot-i_vector
                direction[0]=direction[0]-(boxl*np.round(direction[0]/boxl))
                direction[1]=direction[1]-(boxl*np.round(direction[1]/boxl))
                direction[2]=direction[2]-(boxl*np.round(direction[2]/boxl))
                dist=np.linalg.norm(direction)
                if(dist<cutoff):
                    #can set notused = 1 here. Need to ponder
                    
                    return False
        return True
                

In [15]:
def parse(f_name):
    config=[]
    with open(f_name,'r') as f:
        temp=[]
        for i in f.readlines():
            i=str(i)
            #print(i)
            if("begin" in i):
                temp=[]
            elif("end" in i):
                config.append(temp)
                break # need to remove for a whole file
            elif(i.startswith("atom")):
                a=i.split()
                temp.append(Atom(a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8],a[9]))
    #for j in config:
    #    for i in j:
    #        print(i.element,type(i.element))
    return config                  

In [4]:
def pH(proton_count,boxl,Na):
    print("Proton Count = "+str(proton_count))
    n=proton_count/Na
    vol=boxl**3
    Molarity=n/vol
    print(Molarity)
    return -1*(np.log10(Molarity))

In [34]:
def main():
    cutoff=2.173185 # needs to be atomic units since the coordinates are in atomic units
    boxl_atomic=23.465110
    Na=6.0221409e+23
    atomic_to_angstron=0.5291772083
    boxl_angstron=boxl_atomic*atomic_to_angstron
    boxl_meters=boxl_angstron**-8
    config=parse("../training-set/input.data")
    print("config size",np.shape(config))
    for ind,j in enumerate(config):
        proton_count=0
        for ind2,i in enumerate(j):
            if(i.element=="H"):
                #print("H at"+str(ind2))
                if(i.scan_vicinity(j,cutoff,boxl_angstron)):
                    proton_count=proton_count+1
                    print("atom number %d is a proton and the element is %s"%(ind2 ,i.element))
                    print(i.x,i.y,i.z)
        print("pH of config-"+str(ind)+" = "+str(pH(proton_count,boxl_meters,Na)))

In [35]:
if __name__ == "__main__":
    main()

config size (1, 192)
atom number 50 is a proton and the element is H
20.5757 16.2917 16.9078
atom number 190 is a proton and the element is H
10.61 17.8367 7.87629
Proton Count = 2
599.5818662961775
pH of config-0 = -2.7778484896088695
